In [1]:
import tensorflow_datasets as tfds
import tensorflow as tf

In [2]:
dataset, info = tfds.load("tf_flowers",
                          as_supervised=True,
                          with_info=True)

Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]

Dataset tf_flowers downloaded and prepared to /root/tensorflow_datasets/tf_flowers/3.0.1. Subsequent calls will reuse this data.


In [3]:
dataset_size = info.splits["train"].num_examples
class_names = info.features["label"].names
n_classes = info. features["label"].num_classes

In [5]:
test_set_raw, valid_set_raw, train_set_raw = tfds.load("tf_flowers",
                        split=["train[:10%]",
                               "train[10%:25%]",
                               "train[25%:]"],
                        as_supervised=True)

In [6]:
def preprocess(image,label):
    resized_image = tf.image.resize(image, [224,224])
    final_image = tf.keras.applications.xception.preprocess_input(resized_image)
    return final_image, label


In [7]:
batch_size = 32
train_set = train_set_raw.shuffle(1000)
train_set = train_set.map(preprocess).batch(batch_size).prefetch(1)
valid_set = valid_set_raw.map(preprocess).batch(batch_size).prefetch(1)
test_set = test_set_raw.map(preprocess).batch(batch_size).prefetch(1)

In [8]:
base_model = tf.keras.applications.xception.Xception(weights="imagenet",
                                            include_top=False)


avg = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
output = tf.keras.layers.Dense(n_classes, activation="softmax")(avg)


model = tf.keras.Model(inputs=base_model.input, outputs=output)

83683744/83683744 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


In [9]:
for layer in base_model.layers:
  layer.trainable = False

In [11]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.2,
                                    momentum=0.9,
                                    decay=0.01)

model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])


history = model.fit(train_set,
                    epochs=5,
                    validation_data=valid_set)

Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/optimizers/base_optimizer.py:33: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


86/86 ━━━━━━━━━━━━━━━━━━━━ 39s 231ms/step - accuracy: 0.6937 - loss: 1.7500 - val_accuracy: 0.8421 - val_loss: 1.4094
Epoch 2/5
86/86 ━━━━━━━━━━━━━━━━━━━━ 18s 139ms/step - accuracy: 0.8900 - loss: 0.7576 - val_accuracy: 0.8312 - val_loss: 1.8795
Epoch 3/5
86/86 ━━━━━━━━━━━━━━━━━━━━ 21s 149ms/step - accuracy: 0.8966 - loss: 0.8154 - val_accuracy: 0.8584 - val_loss: 1.4228
Epoch 4/5
86/86 ━━━━━━━━━━━━━━━━━━━━ 21s 151ms/step - accuracy: 0.9372 - loss: 0.3674 - val_accuracy: 0.8603 - val_loss: 1.1746
Epoch 5/5
86/86 ━━━━━━━━━━━━━━━━━━━━ 20s 147ms/step - accuracy: 0.9523 - loss: 0.2306 - val_accuracy: 0.8566 - val_loss: 1.2976


In [12]:
for layer in base_model.layers:
  layer.trainable = True

In [14]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01,
                                    momentum=0.9,
                                    decay=0.001)

model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])

history = model.fit(train_set,
                    epochs=10,
                    validation_data=valid_set)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/optimizers/base_optimizer.py:33: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


86/86 ━━━━━━━━━━━━━━━━━━━━ 81s 520ms/step - accuracy: 0.7825 - loss: 0.9077 - val_accuracy: 0.8893 - val_loss: 0.4471
Epoch 2/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 53s 460ms/step - accuracy: 0.9641 - loss: 0.1166 - val_accuracy: 0.9020 - val_loss: 0.3779
Epoch 3/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 41s 458ms/step - accuracy: 0.9850 - loss: 0.0474 - val_accuracy: 0.9093 - val_loss: 0.3007
Epoch 4/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 41s 457ms/step - accuracy: 0.9906 - loss: 0.0252 - val_accuracy: 0.9238 - val_loss: 0.2841
Epoch 5/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 41s 457ms/step - accuracy: 0.9960 - loss: 0.0170 - val_accuracy: 0.9292 - val_loss: 0.2887
Epoch 6/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 42s 464ms/step - accuracy: 0.9958 - loss: 0.0157 - val_accuracy: 0.9220 - val_loss: 0.3795
Epoch 7/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 41s 463ms/step - accuracy: 0.9947 - loss: 0.0175 - val_accuracy: 0.9093 - val_loss: 0.3716
Epoch 8/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 40s 458ms/step - accuracy: 0.9953 - loss: 0.0103 - val_accuracy: 0.914